# MNIST Handwritten digit recognition

In [1]:
#Importing necessary libraries

import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import Conv2D,Input, Lambda, Dense, Flatten, Dropout, MaxPool2D
from keras.models import Model
from keras.optimizers import Adam
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
%matplotlib inline

Using TensorFlow backend.


In [2]:
#Reading the data

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
train_data.columns

Index(['label', 'pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5',
       'pixel6', 'pixel7', 'pixel8',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=785)

# Using ANN

In [3]:
from sklearn.model_selection import train_test_split
y = train_data['label']
train_data = train_data.drop(['label'],axis=1)
x_train,x_test,y_train,y_test = train_test_split(train_data,y,test_size=0.1,random_state = 42)

In [4]:
y_train.shape,x_train.shape

((37800,), (37800, 784))

In [5]:
#Normalizing the data

x_train = x_train/255.0
x_test = x_test/255.0

In [12]:
#Building the ANN model

model = Sequential()
model.add(Dense(256, input_shape=(784,), activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(100, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(10, activation="softmax"))

In [13]:
#Compiling with the model parameters

model.compile(Adam(lr=0.01),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [16]:
#Fit the model to our data

op = model.fit(x_train, y_train, epochs=10)

Epoch 1/10
37800/37800 [==============================] - 3s 79us/step - loss: 0.2312 - accuracy: 0.9478
Epoch 2/10
37800/37800 [==============================] - 3s 78us/step - loss: 0.2260 - accuracy: 0.9502
Epoch 3/10
37800/37800 [==============================] - 3s 80us/step - loss: 0.2109 - accuracy: 0.9543
Epoch 4/10
37800/37800 [==============================] - 3s 75us/step - loss: 0.2157 - accuracy: 0.9539
Epoch 5/10
37800/37800 [==============================] - 3s 82us/step - loss: 0.2208 - accuracy: 0.9526
Epoch 6/10
37800/37800 [==============================] - 3s 78us/step - loss: 0.2040 - accuracy: 0.9552
Epoch 7/10
37800/37800 [==============================] - 3s 79us/step - loss: 0.2168 - accuracy: 0.9553
Epoch 8/10
37800/37800 [==============================] - 3s 79us/step - loss: 0.2258 - accuracy: 0.9556
Epoch 9/10
37800/37800 [==============================] - 3s 79us/step - loss: 0.1995 - accuracy: 0.9572
Epoch 10/10
37800/37800 [==============================

In [91]:
frIndex = pd.read_csv('sample_submission.csv')

In [34]:
res = model.predict(test_data)
res = np.argmax(res,axis=1)

In [35]:
ans = list(zip(frIndex.ImageId,res))
ans = pd.DataFrame(ans,columns = ['ImageId','Label'])
ans.to_csv('BasicAnn.csv',index = False)

# Using CNN

In [72]:
#Reading the data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [73]:
#normalizing data
train_images=train_images.reshape(60000, 28, 28, 1)
train_images=train_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0

In [75]:
train_images.shape

(60000, 28, 28, 1)

In [113]:
kernel_size = (3,3)
pool_size = (2,2)
first_filters = 32
dropout_conv = 0.3
IMAGE_SIZE = 28

In [87]:
#Constructing a simple CNN model

model = Sequential()
model.add(Conv2D(first_filters,kernel_size,activation='relu',input_shape = (IMAGE_SIZE,IMAGE_SIZE,1))) #1 if the images are gray. 3 if images are RGB
model.add(Conv2D(first_filters,kernel_size,activation='relu'))
model.add(Conv2D(first_filters,kernel_size,activation='relu'))
model.add(MaxPool2D(pool_size=pool_size))
model.add(Dropout(dropout_conv))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(dropout_dense))
model.add(Dense(10, activation='softmax'))

model.compile(Adam(lr=0.0001), loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_44 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 24, 24, 32)        9248      
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 22, 22, 32)        9248      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 11, 11, 32)        0         
_________________________________________________________________
dropout_26 (Dropout)         (None, 11, 11, 32)        0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 3872)              0         
_________________________________________________________________
dense_22 (Dense)             (None, 128)             

In [89]:
#Fitting the model with our data along with necessary callbacks

filepath = 'CustomCNNMnist.h5'
es = EarlyStopping(monitor='accuracy', mode='max', verbose=1, patience=2)
reduce_lr = ReduceLROnPlateau(monitor='accuracy',factor=0.5, patience=3, verbose=1, mode='max',min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath,monitor='val_accuracy',verbose=1, save_best_only=True,mode='max')

fitting = model.fit(train_images,train_labels,epochs=30,callbacks = [reduce_lr,checkpoint,es])

Epoch 1/30
60000/60000 [==============================] - 7s 109us/step - loss: 0.0625 - accuracy: 0.9805
Epoch 2/30
 1504/60000 [..............................] - ETA: 6s - loss: 0.0528 - accuracy: 0.9827

C:\Users\ImperiousPawan\Anaconda3\envs\tensorflow\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_accuracy available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


60000/60000 [==============================] - 6s 108us/step - loss: 0.0536 - accuracy: 0.9838
Epoch 3/30
60000/60000 [==============================] - 6s 107us/step - loss: 0.0458 - accuracy: 0.9852
Epoch 4/30
60000/60000 [==============================] - 6s 107us/step - loss: 0.0406 - accuracy: 0.9875
Epoch 5/30
60000/60000 [==============================] - 6s 107us/step - loss: 0.0362 - accuracy: 0.9882
Epoch 6/30
60000/60000 [==============================] - 6s 107us/step - loss: 0.0322 - accuracy: 0.9897
Epoch 7/30
60000/60000 [==============================] - 6s 108us/step - loss: 0.0297 - accuracy: 0.9909
Epoch 8/30
60000/60000 [==============================] - 6s 107us/step - loss: 0.0282 - accuracy: 0.9913
Epoch 9/30
60000/60000 [==============================] - 7s 109us/step - loss: 0.0251 - accuracy: 0.9920
Epoch 10/30
60000/60000 [==============================] - 6s 108us/step - loss: 0.0239 - accuracy: 0.9925
Epoch 11/30
60000/60000 [==============================]

In [95]:
test_data = np.array(test_data)
test_data = test_data.reshape(28000,28,28,1)

In [96]:
cnnres = model.predict(test_data)

In [98]:
cnnres = np.argmax(cnnres,axis = 1)
ans = list(zip(frIndex.ImageId,cnnres))
ans = pd.DataFrame(ans,columns = ['ImageId','Label'])
ans.to_csv('BasicCnn.csv',index = False)

# Using ML algorithms

KNN Classifier

In [101]:
y = train_data['label']
train_data = train_data.drop(['label'],axis=1)/255.0
test_data = test_data/255.0

In [102]:
train_data.shape,test_data.shape

((42000, 784), (28000, 784))

In [103]:
train_data.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(train_data,y,test_size = 0.1,random_state = 42)

In [110]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(x_train, y_train)

KNeighborsClassifier()

In [112]:
neigh.score(x_train,y_train)

0.9782275132275132

In [111]:
neigh.score(x_test,y_test)

0.9654761904761905

Gaussian naive bayes classifier

In [6]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(x_train, y_train)

GaussianNB()

In [7]:
gnb.score(x_train,y_train),gnb.score(x_test,y_test)

(0.5661111111111111, 0.5671428571428572)